In [53]:
from posixpath import join
from astropy.io.fits import hdu
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
from ciao_contrib.runtool import search_csc 

from astropy.io import fits 
from fields import fields 
from tqdm import tqdm

In [54]:
df = pd.read_csv('../source_list/all_cv_catalogue.csv')
df = df.replace(' ' , '_' , regex=True)


In [55]:
df['Type'].value_counts()

Candidate               3491
CataclyV                3389
N                       1657
Candidate_CV             891
known_CV                 829
DN                       440
Cataclysmic_Variable     274
Known_CV                 240
Candidate_Nova           210
AntiNova                   1
Name: Type, dtype: int64

In [56]:
df['Type'].unique()

array(['CataclyV', 'Candidate_CV', nan, 'known_CV', 'N', 'Candidate',
       'Candidate_Nova', 'Known_CV', 'DN', 'AntiNova',
       'Cataclysmic_Variable'], dtype=object)

In [62]:
df_select = df[df['Type'].isin(['CataclyV', 'Candidate_CV', 'known_CV', 'N', 'Candidate',
       'Candidate_Nova', 'Known_CV', 'DN', 'AntiNova',
       'Cataclysmic_Variable'])].reset_index(drop=True)

In [63]:
df_select

,Name,Disc. Date,Mag.,R.A.,Dec.,Type
0,1RXS_J013308.9+383218,2009/10/27,NaN,01:33:08.72,+38:32:17.2,CataclyV
1,1RXS_J044027.0+023300,2009/02/19,NaN,04:40:27.14,+02:33:01.0,CataclyV
2,1RXS_J221343.9+173301,2009/09/17,NaN,22:13:44.03,+17:32:51.9,CataclyV
3,2MASS_J00153821+2636574,2009/09/18,NaN,00:15:38.2393,+26:36:56.815,CataclyV
4,2MASS_J23255145-0140232,2009/11/16,NaN,23:25:51.4767,-01:40:23.646,CataclyV
...,...,...,...,...,...,...
11417,ZTF18abwurjm,NaN,19.7,20:07:54.82,+03:27:38.1,DN
11418,ZTF18aceglmy,NaN,19.6,19:49:47.59,+03:13:05.2,DN
11419,ZTF18acrultm,NaN,NaN,05:35:49.47,-03:32:51.3,CataclyV
11420,ZTF19acecsfi,NaN,NaN,07:59:30.81,+44:16:21.5,Candidate_CV


In [50]:
ra = df_select['R.A.']
dec = df_select['Dec.']
name = df_select['Name']
num_obs = [] 
offset = []


In [ ]:
type = df_select['Type']
for index in tqdm(range(len(df_select))):
    try :
        s = search_csc(
                        radunit="arcsec", 
                        columns=fields, 
                        bands="broad",
                        clobber="yes" ,
                        radius= 10 , 
                        pos= str(ra[index])+','+str(dec[index])  ,
                        outfile='temp.csv')
                    #sys('search_csc outfile=trial.csv radunit=arcsec columns="SOS,SOP,SOV , o.gti_obs m.flux_aper_b" bands=broad clobber=yes radius=1 pos="65.428058,32.907468"')
                    #print(s)
                    
        data = pd.read_csv('temp.csv', delimiter='\t' , comment='#')
        data = data[data['match_type']=='u          ']
        data = data[data['instrument']=='ACIS']
        data.index.name= 'index'
        #f_name_fits = data_fits['A_NAME']
        n = len(data)
        num_obs.append(n)
        if(n>0):
            print('Match found --' , len(data))
            data.to_csv('../data/all_data/CV_v2_all/'+name[index]+'.csv')
    except:
        print('Manual ispection needed :' , name[index])
    #data.to_csv('temp_v2.csv')
df_updated = df_select.copy()
df_updated.insert(5 , 'num_obs' , num_obs)

In [33]:
df_updated.to_csv('../source_list/CV_updated.csv')